## MLflow's Model Registry

In [1]:
from mlflow.tracking import MlflowClient


#MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
MLFLOW_TRACKING_URI = "http://ec2-3-27-119-22.ap-southeast-2.compute.amazonaws.com:5000"

In [2]:
import os

# Remove any partial credentials
for key in ['AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY', 'AWS_SESSION_TOKEN']:
    os.environ.pop(key, None)

# CRITICAL: Tell MLflow to NOT pass explicit credentials to boto3
os.environ['MLFLOW_S3_ENDPOINT_URL'] = ''  # Empty to use default
os.environ['AWS_DEFAULT_REGION'] = 'ap-southeast-2'

# Force reload of MLflow modules to pick up new environment
import sys
if 'mlflow' in sys.modules:
    # Remove mlflow from cache
    modules_to_remove = [k for k in sys.modules.keys() if k.startswith('mlflow')]
    for module in modules_to_remove:
        del sys.modules[module]

# Now import MLflow fresh
from mlflow.tracking import MlflowClient


client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

run_id = "c83577e800dc4c528d27d97b5a20cd65"
artifacts = client.list_artifacts(run_id)
print(f"✓ Success! Artifacts: {artifacts}")

/home/ubuntu/anaconda3/lib/python3.9/site-packages/boto3/compat.py:84: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


PartialCredentialsError: Partial credentials found in explicit, missing: aws_secret_access_key

In [3]:
import os
import boto3
from mlflow.tracking import MlflowClient

# Clear credentials
for key in ['AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY', 'AWS_SESSION_TOKEN']:
    os.environ.pop(key, None)

os.environ['AWS_DEFAULT_REGION'] = 'ap-southeast-2'

# Monkey-patch MLflow's S3 client creation
import mlflow.store.artifact.s3_artifact_repo as s3_repo

original_get_s3_client = s3_repo._get_s3_client

def patched_get_s3_client(*args, **kwargs):
    """Force boto3 to use IAM role by not passing explicit credentials"""
    # Remove any explicit credentials from kwargs
    kwargs.pop('aws_access_key_id', None)
    kwargs.pop('aws_secret_access_key', None)
    kwargs.pop('aws_session_token', None)
    
    # Just create a simple S3 client with IAM role
    return boto3.client('s3', region_name=os.environ.get('AWS_DEFAULT_REGION'))

# Apply the patch
s3_repo._get_s3_client = patched_get_s3_client

# Now use MLflow

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

run_id = "c83577e800dc4c528d27d97b5a20cd65"
artifacts = client.list_artifacts(run_id)
print(f"✓ Success! Artifacts: {artifacts}")

✓ Success! Artifacts: [<FileInfo: file_size=None, is_dir=True, path='preprocessor'>]


In [3]:
import os
import boto3
from mlflow.tracking import MlflowClient
from botocore.credentials import InstanceMetadataProvider, InstanceMetadataFetcher

# 1. Check what boto3 sees
print("=" * 50)
print("1. BOTO3 CREDENTIALS CHECK")
print("=" * 50)
session = boto3.Session()
credentials = session.get_credentials()
if credentials:
    print(f"Access Key: {credentials.access_key[:10]}... (from {credentials.method})")
    print(f"Secret Key: {'***PRESENT***' if credentials.secret_key else '***MISSING***'}")
    print(f"Token: {'***PRESENT***' if credentials.token else 'None'}")
else:
    print("No credentials found!")

# 2. Check MLflow's view of the artifact
print("\n" + "=" * 50)
print("2. MLFLOW ARTIFACT URI CHECK")
print("=" * 50)
#MLFLOW_TRACKING_URI = "http://localhost:5000"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
run_id = "c83577e800dc4c528d27d97b5a20cd65"  # your run_id
run = client.get_run(run_id)
artifact_uri = run.info.artifact_uri
print(f"Artifact URI: {artifact_uri}")

# Check if credentials are in the URI
if '@' in artifact_uri:
    print("⚠️  WARNING: Credentials embedded in artifact URI!")
    # Parse it
    parts = artifact_uri.split('@')
    print(f"Credential part: {parts[0]}")

# 3. Try to create artifact repo and see what it uses
print("\n" + "=" * 50)
print("3. ARTIFACT REPOSITORY CREATION")
print("=" * 50)
try:
    from mlflow.store.artifact.artifact_repository_registry import get_artifact_repository
    
    # This is what MLflow does internally
    artifact_repo = get_artifact_repository(artifact_uri)
    print(f"Artifact repo type: {type(artifact_repo)}")
    
    # Check if it's S3 and what client it's using
    if hasattr(artifact_repo, 's3_client'):
        print("Has s3_client attribute")
        s3_client = artifact_repo.s3_client
        # Try to get credentials from the client
        if hasattr(s3_client, '_request_signer'):
            signer_creds = s3_client._request_signer._credentials
            print(f"S3 client credentials method: {signer_creds.method if signer_creds else 'None'}")
            print(f"Access key present: {bool(signer_creds.access_key) if signer_creds else False}")
            print(f"Secret key present: {bool(signer_creds.secret_key) if signer_creds else False}")
            
except Exception as e:
    print(f"Error creating artifact repo: {e}")
    import traceback
    traceback.print_exc()

# 4. Try list_artifacts and catch the detailed error
print("\n" + "=" * 50)
print("4. LISTING ARTIFACTS (with detailed error)")
print("=" * 50)
try:
    artifacts = client.list_artifacts(run_id)
    print(f"✓ Success! Found {len(artifacts)} artifacts")
except Exception as e:
    print(f"✗ Error: {e}")
    import traceback
    print("\nFull traceback:")
    traceback.print_exc()

1. BOTO3 CREDENTIALS CHECK
Access Key: ASIAZU3E3A... (from iam-role)
Secret Key: ***PRESENT***
Token: ***PRESENT***

2. MLFLOW ARTIFACT URI CHECK
Artifact URI: s3://mlflow-artifacts-remote31/mlflow-artifacts/c83577e800dc4c528d27d97b5a20cd65/artifacts

3. ARTIFACT REPOSITORY CREATION
Artifact repo type: <class 'mlflow.store.artifact.s3_artifact_repo.S3ArtifactRepository'>

4. LISTING ARTIFACTS (with detailed error)
✓ Success! Found 1 artifacts


### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [4]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote31/mlflow-artifacts/21', creation_time=1764262037079, experiment_id='21', last_update_time=1764262037079, lifecycle_stage='active', name='my-experiment9', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote31/mlflow-artifacts/20', creation_time=1764261093983, experiment_id='20', last_update_time=1764261093983, lifecycle_stage='active', name='my-experiment8', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote31/mlflow-artifacts/18', creation_time=1764260523694, experiment_id='18', last_update_time=1764260523694, lifecycle_stage='active', name='my-experiment7', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote31/mlflow-artifacts/17', creation_time=1764260375248, experiment_id='17', last_update_time=1764260375248, lifecycle_stage='active', name='my-experiment6', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote31/mlflow-artifacts/15', creation_t

In [5]:
client.create_experiment(name="my-experiment10")

'22'

Let's check the latest versions for the experiment with id `1`...

In [6]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='6',
    filter_string="metrics.rmse < 10",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [7]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: c83577e800dc4c528d27d97b5a20cd65, rmse: 6.4133
run id: 764260bfa5ec4f84a9b9e462b99a4995, rmse: 6.4795
run id: e540a77b84084cc6a9ea7093b8ad0474, rmse: 6.8793
run id: 28d0aa7a24c44012a3225f006e2fb9ef, rmse: 7.3460
run id: 4e7b2218e7164736a0651ad8fa3802ea, rmse: 7.5920


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [8]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [9]:
run_id = "28d0aa7a24c44012a3225f006e2fb9ef"
artifacts = mlflow.artifacts.list_artifacts(run_id=run_id)

for artifact in artifacts:
    print(f"Path: {artifact.path}, Is dir: {artifact.is_dir}")


Path: estimator.html, Is dir: False


In [10]:
run_id = "28d0aa7a24c44012a3225f006e2fb9ef"
# artifact path here is "model" by default with mlflow.sklearn.autolog()
# The artifact path is specified in mlflow.log_artifact, for example, mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2025/11/27 16:49:13 WARNING mlflow.tracking._model_registry.fluent: Run with id 28d0aa7a24c44012a3225f006e2fb9ef has no artifacts at artifact path 'model', registering model based on models:/m-e386894895dc4272a84b228b5ec72519 instead
2025/11/27 16:49:13 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 10
Created version '10' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1764262153127, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1764262153127, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='28d0aa7a24c44012a3225f006e2fb9ef', run_link='', source='models:/m-e386894895dc4272a84b228b5ec72519', status='READY', status_message=None, tags={}, user_id='', version='10'>

In [11]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Staging
version: 10, stage: None


/tmp/ipykernel_3461/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [25]:
model_version = 2
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_3461/3059153048.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1764190143755, current_stage='Production', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='The model version 2 was transitioned to Staging on 2025-11-27', last_updated_timestamp=1764262270239, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='28d0aa7a24c44012a3225f006e2fb9ef', run_link='', source='models:/m-e386894895dc4272a84b228b5ec72519', status='READY', status_message=None, tags={}, user_id='', version='2'>

In [26]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1764190143755, current_stage='Production', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='The model version 2 was transitioned to Production on 2025-11-27', last_updated_timestamp=1764262270828, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='28d0aa7a24c44012a3225f006e2fb9ef', run_link='', source='models:/m-e386894895dc4272a84b228b5ec72519', status='READY', status_message=None, tags={}, user_id='', version='2'>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [14]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [15]:
df = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet")

In [16]:
import boto3
s3 = boto3.client("s3")
print(s3.list_buckets())  # should work without static keys


{'ResponseMetadata': {'RequestId': 'H0PZ31RM3V8WAB23', 'HostId': 'UPIzGIOWHrL+oeg5n9f/LmRyjsD1+3ysa3XuYHX/5zDO+iaBusiglLu0euWqqI/jfNbEEqSHh+2OHSDJzHT/F7m7D+Iz/BEP3Uh88WkUZeA=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'UPIzGIOWHrL+oeg5n9f/LmRyjsD1+3ysa3XuYHX/5zDO+iaBusiglLu0euWqqI/jfNbEEqSHh+2OHSDJzHT/F7m7D+Iz/BEP3Uh88WkUZeA=', 'x-amz-request-id': 'H0PZ31RM3V8WAB23', 'date': 'Thu, 27 Nov 2025 16:49:31 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Buckets': [{'Name': 'mlflow-artifacts-remote31', 'CreationDate': datetime.datetime(2025, 11, 23, 16, 14, 16, tzinfo=tzlocal()), 'BucketArn': 'arn:aws:s3:::mlflow-artifacts-remote31'}], 'Owner': {'ID': '28c664863959232ad824290e29a7cffeba419fb579cbbbb96b43737d9a47349f'}}


/home/ubuntu/anaconda3/lib/python3.9/site-packages/boto3/compat.py:84: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [17]:
run_id = "c83577e800dc4c528d27d97b5a20cd65"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
run = client.get_run(run_id)
print("Artifact URI:", run.info.artifact_uri)

Artifact URI: s3://mlflow-artifacts-remote31/mlflow-artifacts/c83577e800dc4c528d27d97b5a20cd65/artifacts


In [19]:
run_id = "c83577e800dc4c528d27d97b5a20cd65"

client.list_artifacts(run_id)


[<FileInfo: file_size=None, is_dir=True, path='preprocessor'>]

In [20]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/home/ubuntu/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [21]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [22]:
X_test = preprocess(df, dv)

In [23]:
target = "duration"
y_test = df[target].values

In [27]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

/home/ubuntu/anaconda3/lib/python3.9/site-packages/boto3/compat.py:84: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


CPU times: user 592 ms, sys: 113 ms, total: 705 ms
Wall time: 1.61 s


{'rmse': 1652.4988975613737}

In [28]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

/home/ubuntu/anaconda3/lib/python3.9/site-packages/boto3/compat.py:84: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


CPU times: user 499 ms, sys: 29.7 ms, total: 529 ms
Wall time: 701 ms


{'rmse': 1652.4988975613737}

In [29]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)

/tmp/ipykernel_3461/3610659809.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1764194144705, current_stage='Production', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1764262302473, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='28d0aa7a24c44012a3225f006e2fb9ef', run_link='', source='models:/m-e386894895dc4272a84b228b5ec72519', status='READY', status_message=None, tags={}, user_id='', version='4'>